In [ ]:
!bash ./start_vllm_container.sh

In [ ]:
!pip install openai==0.28.1

In [ ]:
import openai

def invoke_vllm(prompt, assist_role=True, stream=True, max_tokens=400):

    # Modify OpenAI's API key and API base to use vLLM's API server.
    openai.api_key = "EMPTY"
    vllm_container_port = 8000
    openai.api_base = f"http://localhost:{vllm_container_port}/v1"

    # List models API
    models = openai.Model.list()
    #print("Models:", models)

    model = models["data"][0]["id"]

    # Completion API
    
    if (assist_role):
        prompt = f"<|prompter|>{prompt}</s><|assistant|>"

    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        echo=False,
        n=1,
        stream=stream,
        max_tokens=max_tokens,
        #ignore_eos=False,
        #temperature=0.1,
        temperature=0,
        #top_p=0.95,
        #stop=["</s>"],
     )
    if stream:
        output = ""
        for c in completion:
            #print(c)
            d = c["choices"][0]["text"]
            output += d
            print(d, end='', flush=True)      
    else:
        output = completion["choices"][0]["text"]
        print(output)
    return output

In [ ]:
prompt = "What are the main challenges to support a long context for LLM?"
result = invoke_vllm(prompt)

In [ ]:
with open("../example_long_ctx.txt", "r") as fin:
    task_instruction = fin.read()
    task_instruction = task_instruction.format(
        my_question="please tell me how does pgvector help with Generative AI and give me some examples."
    )
prompt = f"<|prompter|>{task_instruction}</s><|assistant|>"
invoke_vllm(prompt)